# XRD-CT reconstruction from flyscans
# !!!!! run this notebook inside the Tomography / Tomorec / GPU environment !!!!!

This notebook will do a simple tomo resontruction for a XRD-CT map (omega-translation flyscan). It requires data from the `.h5` masterfile and the `_pilatus_integrated.h5` file.

### Load data and plot average XRD for the map

In [ ]:
%pylab inline
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt

try:
    import tomopy
except ModuleNotFoundError as err:
    raise Exception('tomopy not found: try changing your server to "Tomography/Tomorec/GPU": in File->Hub control panel')
    
# Define scan location:
scan = 'xxxx'
folder = '/data/visitors/danmax/xxxx/xxxx/raw/xxxx/'

#radial_roi = np.arange(190,260)

fh = h5py.File(os.path.join(folder, 'scan-%s.h5' %scan), 'r')
print("Scan commmand used: {0}".format(fh['entry/title'][()].decode('utf-8')))
macroname = fh['entry/title'][()].decode('utf-8').split()
nominalXsteps = int(macroname[4])+1
nominalYsteps = int(macroname[8])+1

print('original scan dimensions are '+str(nominalXsteps)+'x'+str(nominalYsteps)+' points')

# first motor in meshct is the fast one
y = fh['/entry/instrument/ea_sams2_ry/value'][:]
#y = fh['/entry/instrument/ea_ppm3_2/value'][:]
x = fh['/entry/instrument/sam_y/value'][:]

#if the scan was interrupted, get the actual dimension in X, to build a map of the correct aspect ratio
actualXsteps = int(len(x)/nominalYsteps)
print('originally '+str(nominalXsteps)+' rotation angles were plannes')
print('actually '+str(actualXsteps)+' rotation angles are found in the file')


fh = h5py.File(os.path.join(folder.replace('raw', 'process/azint'), 'scan-%s_pilatus_integrated.h5' %scan), 'r')

sig1 = np.mean(fh['I'][:], axis=0)

#Determine if its 2theta or q
try:
    r_unit = fh['2th']
    tth = True
except KeyError:
    r_unit = fh['q']
    tth = False

print("Mean of all XRD patterns:")
plt.figure(figsize=(12,12))
plt.plot(sig1)
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')

### Reshape XRD data for tomo reconstruction

In [ ]:
nazim = fh['I'].shape[1]
diff2d = np.reshape(fh['I'], (nominalYsteps, nominalXsteps, nazim))
angles2d = np.reshape(y, ((nominalYsteps, nominalXsteps)))
angles2d = angles2d.transpose(1,0)
proj = diff2d.transpose(0,2,1)
theta = np.radians(angles2d[0])

### Plot a sinogram at a given q-point
For visual confirmation that everything is OK

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(proj[:,470,:], cmap='viridis')#, extent=[x[0], x[-1], y[0], y[-1]])
plt.xlabel('ea_ppm3_3 / mm')
plt.ylabel('pcap_mean (ea_ppm3_2) / mm')
plt.colorbar()
plt.show()
#usefull cmaps:
# 'CMRmap', 'GnBu', 'OrRd', PuBu', 'PuBuGn', 'PuOr', 'PuRd',
# 'RdPu', 'RdYlGn', 'Reds', 'Spectral', 'Wistia', 'YlOrRd', 'autumn',
# 'cividis', 'coolwarm', 'copper', 'gist_earth', 'gist_heat', 'gnuplot',
# 'hot', 'inferno', 'jet', 'magma', 'nipy_spectral', 'ocean', 'plasma', 'rainbow',
# 'seismic', 'spring', 'summer', 'viridis', 'winter'

In [ ]:
#add padding
proj=tomopy.misc.morph.pad(proj, axis=2, mode='edge')


#find rotation center (pick an algorithm)
#rot_center = tomopy.find_center(proj, theta)
rot_center = tomopy.find_center_pc(proj[0], proj[-1], tol=0.5)
#rot_center = tomopy.find_center_vo(proj)

print('rotation center is {}'.format(rot_center))

plt.figure(figsize=(12,12))
plt.imshow(proj[:,100,:], cmap='viridis')
plt.show()

In [ ]:
#tomo reconstruction
recon = tomopy.recon(proj, theta, center=rot_center, algorithm='gridrec', filter_name='hamming')

print(recon.shape)

#add a circular mask
recon = tomopy.circ_mask(recon, axis=0, ratio=0.95)

plt.figure(figsize=(12,12))
plt.imshow(recon[422, :,:], cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(recon[500, :,:], cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
#Saving a PyMCA-usable H5 file

fh = os.path.join(folder.replace('raw', 'process/ct'), 'scan-%s_recon.h5' %scan)

out_path = folder.replace('raw', 'process/ct')
if not os.path.exists(out_path):
    print(f'Creating folder: {out_path}')
    os.makedirs(out_path)

print(fh)

out= h5py.File(fh, 'w')
out.create_dataset('/reconstrucred/gridrec', data=recon.transpose(1,2,0))
if tth:
    out.create_dataset('2th', data=r_unit)
else:
    out.create_dataset('q', data=r_unit)
out.close()

print('h5 file is saved')